goal is to get 2022 technics sets inventory listed and calculate super-set and corelation for all set to super-set<br>
to do:
* get id of technic theme from GET /api/v3/lego/themes/ and save a it
* get list of sets from GET /api/v3/lego/sets/ filtered by paramters: technic theme id and year 2022 
* for each set get inventory from GET /api/v3/lego/sets/{set_num}/parts/ and store it in table
* group by set_num and part_num - parts could be repeated in inventory due to diffence in color
* build super-set(max for each part)
* calculate corelation for each of starting sets with super-set

In [37]:
import requests
import json

keys = json.load(open('keys.json'))
API_KEY = '/?key=' + keys['API_KEY']
api_url = 'https://rebrickable.com/api/v3/'

In [41]:
themes_url = api_url + 'lego/themes' + API_KEY
response = requests.get(themes_url).json()
themes = response['results']

for theme in themes:
    if theme['name'] == 'Technic':
        technic_theme_id = str(theme['id'])

technic_theme_id


'1'

In [42]:

theme_id = technic_theme_id
min_year = '2022'
max_year = '2022'
sets_url = api_url + 'lego/sets' + API_KEY + \
    '&theme_id=' + theme_id + '&min_year=' + min_year + '&max_year=' + max_year
response = requests.get(sets_url).json()
sets = response['results']
sets_list = []

for set in sets:
    sets_list.append(set['set_num'])

sets_list


['30433-1',
 '42130-1',
 '42132-1',
 '42133-1',
 '42134-1',
 '42135-1',
 '42136-1',
 '42137-1',
 '42138-1',
 '42139-1',
 '42140-1',
 '42141-1',
 '42141-2',
 '42143-1',
 '42144-1',
 '42145-1',
 '66712-1']

In [68]:
set_numbers, part_numbers, quantities = [], [], []

for set in sets_list:
    inventory_parts_url = api_url + 'lego/sets/' + \
        set + '/parts' + API_KEY + '&inc_color_details=0'
    response = requests.get(inventory_parts_url).json()
    inventory_parts = response['results']
    for part in inventory_parts:
        set_numbers.append(set)
        part_numbers.append(part['part']['part_num'])
        quantities.append(part['quantity'])


inventory_part_dictionary = {'set_number': set_numbers,
             'part_number': part_numbers, 'quantity': quantities}
import pandas as pd

df = pd.DataFrame(inventory_part_dictionary)
df

,set_number,part_number,quantity
0,30433-1,50335,1
1,30433-1,53923,1
2,30433-1,11478,2
3,30433-1,32348,2
4,30433-1,32526,2
...,...,...,...
1386,42145-1,32073,27
1387,42145-1,60485,5
1388,42145-1,32013,8
1389,42145-1,6536,27


In [67]:
agg_af = df.groupby(['set_number','part_number']).sum()


quantity
set_number part_number           
30433-1    10928                2
           11478                4
           15462                1
           24116                1
           27940                4
...                           ...
42145-1    92907                1
           98585                1
           99010                1
           99012                3
           99013pat0002         5

[1195 rows x 1 columns]

In [65]:
df[df['set_number'] == '30433-1']


,set_number,part_number,quantity
0,30433-1,50335,1
1,30433-1,53923,1
2,30433-1,11478,2
3,30433-1,32348,2
4,30433-1,32526,2
5,30433-1,62462,1
6,30433-1,89201,4
7,30433-1,42924,8
8,30433-1,32316,3
9,30433-1,10928,2


In [74]:
volvo_set_df = agg_af.query("set_number=='30433-1'")
volvo_set_df

quantity
set_number part_number          
30433-1    10928               2
           11478               4
           15462               1
           24116               1
           27940               4
           32002               3
           32073               3
           32123b              5
           32140               2
           32316               3
           32348               2
           32526               3
           32905               1
           39888               4
           4274                5
           42924               8
           4519                3
           50335               1
           53923               1
           55981               4
           61409               2
           62462               1
           6632                4
           89201               4
           92688               1